In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18622397
paper_name = 'breslow_weissman_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/nmeth.1234-S5.xlsx', sheet_name='Deletion growth rates')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4204 x 10


In [7]:
original_data.head()

,Note,ORF,Gene,Median Growth Rate,Individual growth rate measurements,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,YCR063W,BUD31,< 0.5,< 0.5,NaN,NaN,NaN,NaN,NaN
1,NaN,YDR315C,IPK1,< 0.5,< 0.5,NaN,NaN,NaN,NaN,NaN
2,NaN,YJR073C,OPI3,< 0.5,< 0.5,NaN,NaN,NaN,NaN,NaN
3,NaN,YML013C-A,YML013C-A,0.403,0.4035,NaN,NaN,NaN,NaN,NaN
4,NaN,YOL108C,INO4,0.435,0.4027,0.4676,NaN,NaN,NaN,NaN


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Note, ORF, Gene, Median Growth Rate, Individual growth rate measurements, Unnamed: 5, Unnamed: 6, Unnamed: 7, Unnamed: 8, Unnamed: 9, orf]
Index: []


In [12]:
original_data['data'] = original_data['Median Growth Rate']

In [13]:
original_data.loc[original_data['data']=='< 0.5','data'] = 0
original_data['data'] = original_data['data'].astype(float)

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4179, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [26]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,26
data_type,value
orf,
YAL002W,0.690
YAL004W,1.011
YAL005C,0.992
YAL007C,1.003
YAL008W,1.009


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,26
,data_type,value
gene_id,orf,
2,YAL002W,0.690
1863,YAL004W,1.011
4,YAL005C,0.992
5,YAL007C,1.003
6,YAL008W,1.009


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          26          
data_type        value    valuez
gene_id orf                     
2       YAL002W  0.690 -3.874055
1863    YAL004W  1.011  0.237132
4       YAL005C  0.992 -0.006210
5       YAL007C  1.003  0.134672
6       YAL008W  1.009  0.211517

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18622397...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.88s/it]

Updating the data_modified_on field...
